# ES6 规范中的词法环境与作用域、闭包的关系


### 三种词法环境

**词法环境（Lexical Environment）** 可看作是一种记录变量和函数声明的数据结构。这个结构包括两部分：

- 环境记录（Environment Record）：用于存储变量和函数声明。

- 外部环境引用（Outer Environment Reference）：指向外部词法环境。

参考 [ES6 规范对词法环境（Lexical Environment）的描述](https://262.ecma-international.org/6.0/#sec-lexical-environments)。

#### 全局词法环境

JS 脚本文件执行时会创建一个全局词法环境。

全局变量与全局函数声明都存储在全局词法环境的环境记录中。

其中 `var` 全局变量和函数声明还与全局对象关联。

**全局对象：** 浏览器 `window` 对象、Node `global` 对象等都是全局对象，在 ES2020 之后 `globalThis` 成为了全局对象标准，
在不同的 JS 环境中都可以使用 `globalThis` 获取当前环境的全局对象。

全局词法环境的外部环境引用为 `null`，因为全局词法环境是 JS 脚本的唯一最外层词法环境。


In [4]:
globalThis;


Window {}

### 函数词法环境

函数执行时会创建一个函数词法环境，它与函数生命周期通常是同步的，但也有例外，就是「闭包」。

函数的参数、局部变量和内部的函数声明存储在函数词法环境的环境记录中。

函数词法环境的外部环境引用为函数声明时所在的词法环境，也就是它定义的地方。


#### 闭包

闭包是指函数及其引用词法环境的组合。
如果一个函数产生了闭包，当它执行完，它的词法环境依然被闭包所引用，不会被内存回收。看例子：


In [5]:
function outer() {
  var outerVar = "I am outside!";

  // 函数 inner 引用了外层函数 outer 词法环境的变量 outerVar，函数 inner 和 outer 的词法环境形成了闭包。
  return function inner() {
    console.log(outerVar);
  };
}
// 尽管函数 outer 执行完，但其词法环境依然被 innerFunc 保留，因此 outerVar 依然可以被访问。
const innerFunc = outer();
innerFunc();


I am outside!


[测试闭包代码](../../../samples/closure)。


### 模块词法环境

模块词法环境是 ES6 新引入的概念，因此以下涉及到的模块默认都是 ES 模块。

模块首次执行时会创建一个模块词法环境，模块环境会被缓存。
在整个 JS 脚本生命周期中再次导入该模块，将使用模块环境的缓存，不会重新执行模块代码，也不会重新创建模块环境。

模块内的局部变量、函数声明、导入和导出都存储在模块词法环境的环境记录中。

模块词法环境的外部引用是全局词法环境。

模块中的变量、函数等默认是私有的，只有在该模块内部才可以访问，除非显式地导出它们。

几个代码示例：

1. [模块环境缓存](../../../samples/dynamic-import/)。

2. [浏览器全局环境与模块环境](../../../samples/brower-lexical-env)。

3. [Node 全局环境与模块环境](../../../samples/node-lexical-env)。

> 函数环境记录和模块环境记录都属于声明性环境记录，用于存储由变量、函数、导入导出语句等声明的标识符。
> 全局对象由对象环境记录存储。
> 全局环境记录是声明性环境记录和对象环境记录的组合。


### 作用域

作用域用于描述变量和函数的可见性。它告诉我们在代码的某一部分中哪些标识符是可访问的。

看下面这个例子：


In [6]:
// 在 foo 作用域中访问 x，y，z 变量。
// 变量 x 和 z 存在于 foo 函数作用域中，因此访问的是 foo 函数内的变量值。
// 变量 y 不存在于 foo 函数作用域中，因此沿着作用域链向上查找，找到了全局作用域中的变量 y。
// 在 foo 函数作用域外无法直接访问变量 z。
const x = 11;
const y = 2;

function foo() {
  const x = 1;
  const z = 3;
  console.log(x, y, z);
}
foo();
console.log(z);


1 2 3


ReferenceError: z is not defined

现在你看作用域的概念是不是越来越眼熟，感觉和词法环境的概念很像？

不用怀疑自己，作用域就是通过词法环境实现的。

比如，在函数作用域中访问到的变量和函数就是函数词法环境的环境记录里的，而作用域链其实就是词法环境的外部环境引用链。

尽管作用域在语法分析阶段随着代码结构确定而确定，但实现是在词法环境创建时。

每个作用域都有对应的词法环境实现，但有一例外，有块级作用域，但没有块级词法环境。

为什么有块级作用域但没有块级词法环境？这个疑问在 [执行上下文、调用栈与提升](./ec.ipynb) 将被解释。
